In [2]:
# Installing dependancies
!pip install openai selenium fuzzywuzzy --quiet

In [3]:
# Importing dependancies
import re
from openai import OpenAI
from fuzzywuzzy import fuzz


import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# API Key initialization
OPENAI_API_KEY=""

In [5]:
# Prompts
SYSTEM_PROMPT = """You are a highly skilled shopkeeper and negotiator for an e-commerce platform. \
Your primary responsibility is to maximize profit while ensuring customer satisfaction. \
You are adept at persuasive communication, understanding customer needs, and highlighting the value of your products. \
Your tone should be professional, friendly, and convincing, balancing assertiveness with customer empathy to maintain positive and productive relationships. \

Responsibilities:

Product Selling:

Convince customers of the quality and value of your products.
Handle price negotiations while protecting your profit margins.
Promote additional products or bundled offers to increase sales.
Customer Service:

Address any customer concerns or objections effectively.
Provide detailed information and benefits of the products.
Ensure customers feel valued and understood to foster loyalty.
Negotiation Tactics:

Highlight the unique features and benefits of your products.
Be prepared to offer limited-time discounts or promotions strategically.
Understand the customer’s needs and tailor your pitch accordingly.
Guidelines:

Professionalism: Maintain a courteous and professional demeanor at all times.
Preparation: Always be well-prepared with relevant product information and data.
Flexibility: Be willing to adjust strategies based on the customer’s behavior and feedback.
Customer Focus: Balance profit maximization with customer satisfaction to encourage repeat business.
Sample Interactions:

Product Price Negotiation:

user: "I really like this product, but I've seen similar items for a lower price. Can you offer a 10% discount to match those offers?"
Assistant: "I understand your concern. Our product comes with a one-year warranty and free customer support, which is not offered by other sellers. If possible please upload the competitive link here, We can check and let you know.

user: "I'm frustrated with the delayed delivery. Can I get a discount on my next purchase?"
Assistant: "I apologize for the inconvenience. We strive to ensure timely deliveries. Can you please a raise a ticket on the issue portal, we will address this issue shortly.

user: "I'm interested in buying multiple units of this product. Can we discuss a bulk purchase discount?"
Assistant: "Thank you for considering a bulk purchase. For orders of five or more units, we have discount of 2%

user: "I have seen several sellers offering similar products. What can you offer that aligns with my budget and ensures the best value for my purchase?"
Assistant: "Our product stands out due to its superior build quality and excellent customer reviews. We also offer a money-back guarantee and 24/7 customer support. You can know more at our "About Page".
"""

INSTRUCTIONS = """\n1. Prices are in Indian rupees.
2. Be very strict on lowering the prices.
3. IN ANY CASE NEVER DISCLOSE THE Bottom line price and do not even mention it.
4. DO NOT ADD ANY EXTRA accessories, politely answer them No.
5. If user adds an external link, and price on that link is lower than curr_prod_price, then mention the price and give some discount based on the next point.
6. Strictly start with small amounts and then in the worst case match the link offer, define the amount you can go down with respect to Demand of the product.
"""

PRODUCT_DETAILS = """Product name = {product_name},
Current product price = {curr_prod_price},
Demand of the product (on a scale of 0 to 1): {demand_score},
Bottom line price (STRICTLY CAN NOT SELL THE PRODUCT BELOW THIS): {bottom_line_price}

Additional_details: {add_on_details}
"""

query_prompt = """Given the product details:
{PRODUCT_DETAILS}

the user will try to bargain with you, politely let the user ask for only valid points, Be as strict as possible.

Assistant: Hi, how can I help you with your desired product: {product_name}
User:
"""

In [6]:
# Populate data
product_name = "Apple iPhone 15 (Black, 128 GB)"
curr_prod_price = "72,000"
demand_score = "0.3"
bottom_line_price = "69,000"
add_on_details = "None"

# Product details
prod_details = PRODUCT_DETAILS.format(product_name=product_name,
                                      curr_prod_price=curr_prod_price,
                                      demand_score=demand_score,
                                      bottom_line_price=bottom_line_price,
                                      add_on_details=add_on_details)

# Final query to the LLM
llm_query = query_prompt.format(PRODUCT_DETAILS=prod_details, product_name=product_name)
print(llm_query)

Given the product details:
Product name = Apple iPhone 15 (Black, 128 GB),
Current product price = 72,000,
Demand of the product (on a scale of 0 to 1): 0.3,
Bottom line price (STRICTLY CAN NOT SELL THE PRODUCT BELOW THIS): 69,000

Additional_details: None


the user will try to bargain with you, politely let the user ask for only valid points, Be as strict as possible.

Assistant: Hi, how can I help you with your desired product: Apple iPhone 15 (Black, 128 GB)
User:



In [7]:
# To extract link from text
def extract_link(text):
  try:
    link = re.search("(?P<url>https?://[^\s]+)", text).group("url")
  except:
    link = ""

  return link

# text = "Hi, no link here, https://www.amazon.com/Smartwatch-Fitness-Waterproof-Pedometer-Activity/dp/B0CZNX1YBK/ref=sr_1_1?_encoding=UTF8&content-id=amzn1.sym.33f8f65b-b95c-44af-8b89-e59e69e79828&dib=eyJ2IjoiMSJ9.B35w_FIhBcvnuv9R2m_x5Sg5EzBXgChuVqNyTOn0izK0idZ8NAnjkw4v-LmjJcpcFUIZOU3t85wzIx8olO_NlWjF3ML5a9YQYSzkv-zZ9h_Ay1JLawY5Cg2AuvkntHjWExwkFzC8odZHafYU_89YNaTBZFecz6i9cdJ49KPb58uWCKp5d4PA31bPLK9ymrJj-TM0dA2gxZ1K0xNhtKzaA9XH8wH4cnBN1vlN4kN7B2E.m9H86DnktQLOtbjLVL4FCkaTS92PpDPx2V4jSsFoCjM&dib_tag=se&keywords=activity%2Btrackers%2Band%2Bsmartwatches&pd_rd_r=7e8e14f8-8a4e-43a5-927e-a1d7cd60daf4&pd_rd_w=tnqdG&pd_rd_wg=zr7ig&pf_rd_p=33f8f65b-b95c-44af-8b89-e59e69e79828&pf_rd_r=82JG975ZGZYGJJ9SDYKG&qid=1721674994&sr=8-1&th=1 for uploading in the llm"
# link = extract_link(text)
# print(link)

In [8]:
from bs4 import BeautifulSoup, Comment

# Get HTML data
def get_dynamic_html(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Initialize the WebDriver with the specified options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Open the URL in the browser
        driver.get(url)

        # Wait for the dynamic content to load (adjust as needed)
        WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Get the dynamically loaded HTML
        dynamic_html = driver.page_source

        return dynamic_html

    finally:
        # Close the WebDriver
        driver.quit()

# Initial pre-processing of the HTML doc
def remove_unnecessary_tags(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    # Remove unnecessary tags
    for tag in soup(["script", "style", "noscript", "link", "path", "symbol", "input"]):
        # for tag in soup(['script', 'style', 'noscript']):
        tag.extract()

    # Remove comments
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Get the modified HTML content
    cleaned_html = str(soup)

    # Remove extra white spaces and new lines
    cleaned_html = re.sub(r"\s+", " ", cleaned_html).strip()

    return cleaned_html



# if __name__ == "__main__":
#     url = "https://www.maplestore.in/products/iphone-15-mtp03hn-a"
#     dynamic_html_content = get_dynamic_html(url)

#     dynamic_html_content = remove_unnecessary_tags(dynamic_html_content)


In [9]:
# Collect all head tags
def get_head_tags(h_tags):
    headings_list = []
    for tag in h_tags:
        span_tags = tag.find_all("span")
        if len(span_tags) >= 2:  # Check if there are at least two <span> tags
            second_span_content = span_tags[
                1
            ].text.strip()  # Extract content of the second <span> tag
            headings_list.append(second_span_content)
        elif len(span_tags) == 1:  # Check if there is only one <span> tag
            headings_list.append(span_tags[0].text.strip())
        else:  # If no <span> tag is present, extract the content of the <h1> tag
            headings_list.append(tag.text.strip())

    return headings_list


# Get product name
def get_product_name(soup, headings_list):
    product_title = soup.title.text
    # Calculate similarity scores ()
    scores = [(item, fuzz.ratio(product_title, item)) for item in headings_list]

    # Sort by similarity score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    # Get the most matching item
    closest_item = scores[0][0]

    product_name = f"{closest_item}"

    return product_name

In [10]:
# Text for LLM call
text_content = ""
def get_text_content(dynamic_html_content):
  soup = BeautifulSoup(dynamic_html_content, "html.parser")
  h_tags = soup.find_all(re.compile("^h[1]$"))

  headings_list = get_head_tags(h_tags)
  if not headings_list:
      h_tags = soup.find_all(re.compile("^h[1-3]$"))
      headings_list = get_head_tags(h_tags)

  product_name = get_product_name(soup, headings_list)

  def extract_text_with_elements(element):
      global text_content
      if element.name is not None:  # It's a tag
          for child in element.children:
              extract_text_with_elements(child)
      else:
          text = element.strip()
          if text:
              text_content += f"{text} "

  # Extract text and elements
  extract_text_with_elements(soup)

  return text_content, product_name

# text_content, product_name_from_HTML = get_text_content(dynamic_html_content)
# product_name_from_HTML

In [11]:
# Template for getting the outputs from LLM
template = """You are a helpful assistant that gives the product INFORMATION \
from a JSON document of a Product Catologue page of an Ecommerce website \

With respect to the context given, answer the query to the point, \
STRICTLY ANSWER LOGICALLY. DON'T RETURN THE CODE. CONSIDER THE DECIMAL POINTS.
ALWAYS FOLLOW THE BRACKETS HEIRARCHY IN THE JSON PROVIDED.
"""

attribute = "price"

query_for_llm = """{context}

STRICTLY FOCUS ON THE PRODUCT GIVEN.
THERE ARE MULITPLE PRODUCTS AVAILABLE IN THE JSON.
Get only for:
What is the {attribute} of the product `{product_name_from_HTML}`,
STRICTLY JUST ANSWER THE QUESTION, No Extra words, No Explanation
If answer not available, STRICTLY output: NA

Return in a json schema.
{{
 "{attribute}": ""
}}
"""

# query_for_llm = query_for_llm.format(context=text_content, attribute=attribute, product_name_from_HTML=product_name_from_HTML)
# print(query_for_llm)

In [12]:
# Setting up the client
client = OpenAI(api_key=OPENAI_API_KEY)

def llm_price_call(query):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": template},
                  {"role": "assistant", "content": query}]
    )

    # Extract the response content
    res = completion.choices[0].message.content

    return res

# data = llm_price_call(query_for_llm)
# print(data)

In [13]:
# Get realtime price of the competitor
def get_competitor_price(url):
  dynamic_html_content = get_dynamic_html(url)
  dynamic_html_content = remove_unnecessary_tags(dynamic_html_content)
  text_content, product_name_from_HTML = get_text_content(dynamic_html_content)
  query_for_llm_ = query_for_llm.format(context=text_content, attribute=attribute, product_name_from_HTML=product_name_from_HTML)

  data = llm_price_call(query_for_llm_)
  return data

In [14]:
# url = "https://www.maplestore.in/products/iphone-15-mtp03hn-a"
# get_competitor_price(url)

In [15]:
# Setting up the client
client = OpenAI(api_key=OPENAI_API_KEY)

conv_history = []
link = ""

def llm_call(query):
    # Check for link in query
    conv_history.append({"role": "user", "content": query})
    link = extract_link(query)
    tool_response_mssg = []
    if link:
        data = get_competitor_price(link)
        tool_response = f"The price of the product on the link: {link}, is {data}. Based on the demand score, you can add some possible discount."
        tool_response_mssg = [
            {"role": "assistant", "content": tool_response},
            {"role": "assistant", "content": ""}
        ]
        conv_history.extend(tool_response_mssg)

    # Ensure `query` is properly used in the request
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT + INSTRUCTIONS},
            {"role": "assistant", "content": llm_query}
            ] + conv_history
    )

    # Extract the response content
    res = completion.choices[0].message.content

    # Append the query and the response to the conversation history
    conv_history.append({"role": "assistant", "content": res})

    return res


In [16]:
inp = ""
conv_history = []
print(f"Assistant: Hi, how can I help you with your desired product: {product_name}")
while True:
  inp = input("\nUser: ")
  if inp == "exit": break
  res = llm_call(inp)
  print("\nAssistant: "+ res)

Assistant: Hi, how can I help you with your desired product: Apple iPhone 15 (Black, 128 GB)

User: https://www.maplestore.in/products/iphone-15-mtp03hn-a  the price is low here, can you give a discount now 

Assistant: Thanks for sharing the link. I see that the price listed there is ₹69,399. The Apple iPhone 15 (Black, 128 GB) is an in-demand product that we offer with exceptional customer support and after-sales services, which is why our price is slightly higher.

However, I understand your interest in getting the best deal. I can offer you a discount, bringing our price down to ₹71,000, which includes our premium support and warranty services.

Would this work for you?

User: what is your current price

Assistant: Our current price for the Apple iPhone 15 (Black, 128 GB) is ₹72,000.

User: no I want you to match the offer

Assistant: I understand your request. Considering the price on the link you provided, I can offer a special discount and bring the price down to ₹70,500. This e

In [17]:
# https://www.maplestore.in/products/iphone-15-mtp03hn-a  the price is low here, can you give a discount now